In [218]:
# Bitesnap Parsing
from   collections import OrderedDict
import json
from   pprint import pprint

with open('data/bitesnap/export_06-10-2018_12-29-43.json') as f:
  data = json.load(f)

bitesnap_day = {}

for food in data['entries']:
  d = str(food['eatenAtLocalTime'])[0:8]
 
  # Create Day if we need to...
  if not d in bitesnap_day:
    bitesnap_day[d] = {
      'calories': 0,
      'carbs': 0,
      'netcarbs': 0
    }

  for n in food['nutrients']:
    if n['name'] == 'calories':
      bitesnap_day[d]['calories'] += int(round(n['amount']))
    elif n['name'] == 'totalCarb':
      bitesnap_day[d]['carbs'] += int(round(n['amount']))
      bitesnap_day[d]['netcarbs'] += int(round(n['amount']))
    elif n['name'] == 'dietaryFiber':
      bitesnap_day[d]['netcarbs'] -= int(round(n['amount']))

calories = []
carbs = []
netcarbs = []
for bsd in OrderedDict(sorted(bitesnap_day.items())):
  if int(bsd) >= 20180824:
    calories.append(bitesnap_day[bsd]['calories'])
    carbs.append(bitesnap_day[bsd]['carbs'])
    netcarbs.append(bitesnap_day[bsd]['netcarbs'])

In [235]:
import arrow
import csv
import pandas as pd
import plotly
import plotly.graph_objs as go
from   scipy import signal

# Hmm...
# * Moving avg's sorta not great
# * Better way to label shapes?
# * We want a way to use bar graph for baths..
# * No way to stack y's...


def smoothTriangle(data, degree, dropVals=False):
    triangle=np.array(list(range(degree)) + [degree] + list(range(degree)[::-1])) + 1
    smoothed=[]

    for i in range(degree, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(sum(point)/sum(triangle))
    if dropVals:
        return smoothed
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed


# Load Data
with open('data/qs.csv') as csvfile:
  qsr = csv.reader(csvfile)
  next(qsr)
  qs = list(qsr)

date = []
fasting = []
bath = []
weight = []
for q in qs:
  try:
    d = arrow.get(q[0], 'MM/DD/YY')
  except:
    d = arrow.get(q[0], 'MM/D/YY')
    
  date.append(d)
  
  fasting.append(int(q[1]))
  
  if q[2] == '':
    bath.append(False)
  else:
    bath.append(True)
  
  try:
    weight.append(float(q[3]))
  except:
    weight.append(None)
    

trace_weight = go.Scatter(
  name='Weight',
  x=date, 
  y=weight, 
  mode='lines+markers',
  line={'shape':'spline', 'smoothing':1.3, 'color':'rgba(0,150,200,0.5)'},
)

# https://plot.ly/pandas/time-series/
# https://chrisalbon.com/python/data_wrangling/pandas_moving_average/
data = {'weight': weight}
df = pd.DataFrame(data)
df = df.rolling(window=3).mean()
df['weight'][0] = weight[0]

trace_weight_avg = go.Scatter(
  name='Weight Avg',
  x=date, 
  y=df['weight'],
  line={'shape':'spline', 'smoothing':1.3, 'color':'rgba(0, 150, 200,0.3)'},
  connectgaps=True,
)

trace_fasting = go.Bar(
  name='Fasting',
  x=date, 
  y=fasting,
  marker={'color':'rgba(200, 0, 0,0.2)'},
  yaxis='y2'
)

trace_travel = go.Scatter(
    x=['2018-08-24', '2018-09-05', '2018-09-16', '2018-09-25', '2018-10-02'],
    y=['200','200','200','200', '200'],
    text=['Home',
          'Portland',
          'Home',
          'SF',
          'Home'],
    mode='text',
)

trace_calories = go.Scatter(
  name='Calories',
  x=date, 
  y=calories,
  mode='lines+markers',
  yaxis='y3'
)


trace_carbs = go.Scatter(
  name='Carbs',
  x=date, 
  y=carbs,
  yaxis='y4',
  mode='lines+markers',
)

trace_netcarbs = go.Scatter(
  name='Carbs',
  x=date, 
  y=netcarbs,
  mode='lines+markers',
  yaxis='y4'
)


data = [trace_weight, trace_weight_avg, 
        trace_fasting, trace_calories, trace_carbs, trace_netcarbs, 
        trace_travel
       ]
layout = {
  'title':'Weight Loss',
  'yaxis':dict(
        title='lbs'
    ),
  'yaxis2':dict(
        title='hours',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
  'yaxis3':dict(
    title='cal',
    side='right',
    overlaying='y',
    position=0.9,
  ),
  'yaxis4':dict(
    title='g',
    side='right',
    overlaying='y',
    position=0.8,
  ),
    # to highlight the timestamp we use shapes and create a rectangular
    'shapes': [
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-08-24',
            'y0': 0,
            'x1': '2018-09-05',
            'y1': 1,
            'fillcolor': '#ccbb44',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        },
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-09-05',
            'y0': 0,
            'x1': '2018-09-16',
            'y1': 1,
            'fillcolor': '#99ff00',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        },
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-09-16',
            'y0': 0,
            'x1': '2018-09-25',
            'y1': 1,
            'fillcolor': '#ccbb44',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        },
              {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-09-25',
            'y0': 0,
            'x1': '2018-10-02',
            'y1': 1,
            'fillcolor': '#ff9900',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        },
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-10-02',
            'y0': 0,
            'x1': '2018-10-06',
            'y1': 1,
            'fillcolor': '#ccbb44',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        },
      
      
      # Baths
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-08-27',
            'y0': 0,
            'x1': '2018-08-27',
            'y1': 1,
            'fillcolor': '#000099',
            'opacity': 0.2,
            'line': {
                'width': 3,
            }
        },      
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-09-04',
            'y0': 0,
            'x1': '2018-09-04',
            'y1': 1,
            'fillcolor': '#000099',
            'opacity': 0.2,
            'line': {
                'width': 3,
            }
        },      
        {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': '2018-10-05',
            'y0': 0,
            'x1': '2018-10-05',
            'y1': 1,
            'fillcolor': '#000099',
            'opacity': 0.2,
            'line': {
                'width': 3,
            }
        },      
    ]
}

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
  "data": data,
  "layout": layout
})

# Line under LC, Keto
# Mark in Keto vs Not periods w/ hash
# Net carbs as area graph?, gradient?